This file was made to calculate the precision of Contextual Predictive Mutation Testing, but mutants are selected from clusters instead of from characteristic values.

More in-depth discussion is in Adam Abdalla's thesis.

In [ ]:
import math
import numpy as np
from numpy.random import randint
import pickle
from os import path
import os
from random import sample, seed
import csv
import pandas as pd
from sklearn.cluster import KMeans
import kmeans1d
from hyperopt import hp, fmin, tpe
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
import time
import warnings
import traceback

In [ ]:
# Source: tutorialspoint.com/file-searching-using-python
def find_files(filename, search_path):
   result = []

    # Walking top-down from the root
   for root, _, files in os.walk(search_path):
      if filename in files:
         result.append(os.path.join(root, filename))
   return result


# Gets characteristics and result status data from project file
# created by pitest clustering plugin.
def getProjectDfs(project: str) -> tuple[pd.DataFrame, pd.DataFrame] | int:
    csv_path = "projects/" + project
    charPath = find_files("characteristics.csv", csv_path)

    if charPath:
        charPath = charPath[0]
        data = pd.read_csv(charPath,
                            names=["id", "mutOperator", "opcode", "returnType",
                                    "localVarsCount", "isInTryCatch", "isInFinalBlock",
                                    "className", "methodName", "blockNumber", "lineNumber"],
                            skiprows=1)
        killedPath = find_files("killed.csv", csv_path)

        if killedPath:
            killedPath = killedPath[0]
            results = pd.read_csv(killedPath,
                                    names=["id", "killed", "numTests"],
                                    skiprows=1)
            results = results.drop(columns=["numTests"])
        else:
            print("Could not find killed.csv for project: " + project)
            return -1
    else:
        print("Could not find characteristics.csv for project: " + project)
        return -1

    return data, results


# Uses label encoding and clustering to change numerical data to categorical.
# Also merges data with results.
def dfToCategorical(data: pd.DataFrame, results: pd.DataFrame, parameters: list) -> pd.DataFrame:
    idReduction, localityReduction, n_localVarsClusters = parameters
    # define ordinal encoding
    encoder = LabelEncoder()
    newData = data[["id", "mutOperator", "opcode", "returnType", "localVarsCount",
                    "isInTryCatch", "className", "methodName", "lineNumber"]]
    for col in ["mutOperator", "returnType", "className", "methodName", "id"]:
        newData[col] = encoder.fit_transform(newData[col])

    idClustering = kmeans1d.cluster(np.asarray(newData[["id"]], dtype="int64"), int(math.ceil(len(data) * idReduction)))[0]

    # Categorical locality variable creation.
    newData["className"] = newData["className"].apply(lambda x: x*100000)
    newData["methodName"] = newData["methodName"].apply(lambda x: x*1000)
    localityClustering = kmeans1d.cluster(np.asarray(newData[["className"]], dtype="int64") +
                                          np.asarray(newData[["methodName"]], dtype="int64") +
                                          np.asarray(newData[["lineNumber"]], dtype="int64"), int(math.ceil(len(data) * localityReduction)))[0]

    varsClustering = kmeans1d.cluster(np.asarray(newData[["localVarsCount"]], dtype="int64"), n_localVarsClusters)[0]

    training = data[["id", "mutOperator", "opcode", "returnType", "isInTryCatch"]]
    training["idCluster_id"] = idClustering
    training["localityCluster_id"] = localityClustering
    training["varsCluster_id"] = varsClustering
    training = training.merge(results, how="inner", on="id")

    return training


# Source: https://stackoverflow.com/a/29651514
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result


# Transforms the given dataframe for better clustering as described in Adam's thesis.
# Single characteristic clustering is already done by dfToCategorical.
def preprocessing(data: pd.DataFrame, parameters: list) -> pd.DataFrame:
    idW, mutOpW, opcodeW, retTypeW, locVarsCountW, tryCatchW, localityW = parameters
    normalizedData = normalize(data[["opcode", "varsCluster_id", "isInTryCatch", "localityCluster_id", "idCluster_id"]])
    weightedData = normalizedData.mul([opcodeW, locVarsCountW, tryCatchW, localityW, idW])
    newData = weightedData.join(pd.get_dummies(data["mutOperator"]) * mutOpW)
    newData = newData.join(pd.get_dummies(data["returnType"]) * retTypeW)
    return newData


# Clusters mutants and selects the mutants closest to the center of each cluster.
# May want to change the selection method, considering center selection performed badly in CMT.
def sampleSelector(data: pd.DataFrame, parameters: list, encoder: LabelEncoder, reduction: float) -> list:
    newData = preprocessing(data, parameters)
    clustering = KMeans(n_clusters=int(math.ceil(len(data) * reduction)), n_init=1)
    clusters = clustering.fit_transform(newData)
    clusterCenterIndices = np.argmin(clusters, axis=0)
    mutants = data.loc[clusterCenterIndices]["id"]
    return mutants


# For every possible value of every characteristic, takes all mutants with that value in the sampleDF.
# Calculates the percentage of those mutants which are killed, aka pk-score.
# Overwrites the value in the fullDF with the pk-score. Returns new fullDF.
def  pkScoreTransformer(fullDF: pd.DataFrame, sampleDF: pd.DataFrame, default: float) -> pd.DataFrame:
    for charName in list(fullDF[1:-1]):
        for charVal in list(dict.fromkeys(fullDF[charName].tolist())):
            try:
                percentageKilled = len(sampleDF[(sampleDF[charName] == charVal) & (sampleDF["killed"] == 1)]) / len(sampleDF[sampleDF[charName] == charVal])
            except ZeroDivisionError:
                percentageKilled = default

            fullDF.loc[fullDF[charName] == charVal, charName] = percentageKilled

    return fullDF


# Takes a list of projects, transforms their characteristic data based on parameters
# into a single CPMT-ready dataframe. Trains the given classifier on the dataframe.
def trainAlgorithm(projects: list, classifier: any, parameters: list=[0.05, 0.05, 5, 0.01], reduction: float=0.1, prepParams: list=[13998, 8, 598, 4240, 9505, 15477, 14723]) -> any:
    samplingStop = parameters[-1]
    classificationTraining = []
    for project in projects:
        print("Starting training project: " + project)
        dataframes = getProjectDfs(project)
        if dataframes == -1:
            continue
        data, results = dataframes

        training = dfToCategorical(data, results, parameters[:3])
        print("Starting creation of classification training set for project: " + project)
        start_time = time.time()

        # For every project, a CPMT-ready dataframe is inserted
        # multiple times to produce subsampling. samplingStop controls how many.
        allMutants = []
        mutantsSampleDfs = []
        i = 0
        encoder = LabelEncoder()
        while True:
            print("iteration = ", i)
            i += 1
            print(samplingStop * len(training["id"].tolist()), len(allMutants))
            mutantsSample = sampleSelector(training, prepParams, encoder, reduction)
            allMutants.extend(mutantsSample)
            trainingSample = training[training["id"].isin(mutantsSample)]

            # Default pk-score for if a possible characteristics value has no mutants in selection.
            # Might want to change this by multiplying each killed mutant by the number of mutants
            # in the cluster they represent and dividing by all mutants.
            defaultPK = len(trainingSample[trainingSample["killed"] == 1]) / len(trainingSample)
            trainingInsert = training.copy()
            trainingInsert = pkScoreTransformer(trainingInsert, trainingSample, defaultPK)

            mutantsSampleDfs.append(trainingInsert)

            # Basing it on "allMutants" means lower reductions -> more subsampling.
            # Might want to experiment with different subsampling if classifier gets more complex.
            if samplingStop * len(training["id"].tolist()) < len(allMutants):
                break

        # All dataframes of 1 project get combined.
        classificationTraining.append(pd.concat(mutantsSampleDfs).drop(columns=["id"]))

    print("Done with getting all classification training data.")
    print(time.time() - start_time)

    # Dataframes of all projects get combined.
    classificationTrainingDf = pd.concat(classificationTraining)
    X_train = classificationTrainingDf.drop(columns=["killed"]).values.tolist()
    y_train = classificationTrainingDf["killed"].tolist()

    print("Starting training of classifier.")
    start_time = time.time()
    classifier.fit(X_train, y_train)

    print(time.time() - start_time)
    return classifier


# Predicts the result of every mutant in the given project using CPMT.
def own_predict(classifier: any, project: str, parameters: list=[0.05, 0.05, 5, 0.01], reduction: float=0.1, prepParams: list=[13998, 8, 598, 4240, 9505, 15477, 14723], useSampled: bool=True) -> tuple[np.ndarray, float]:
    samplingStop = parameters[-1]
    print("Starting prediction project: " + project)
    start_time = time.time()
    dataframes = getProjectDfs(project)
    if dataframes == -1:
        return -1
    data, results = dataframes

    # Transformation from raw characteristic data to CPMT-ready data.
    fullDF = dfToCategorical(data, results, parameters[:3])
    sampleMutantsIDs = sampleSelector(fullDF, prepParams, LabelEncoder(), reduction)
    sampleDF = fullDF[fullDF["id"].isin(sampleMutantsIDs)]
    reduction = len(sampleMutantsIDs)/len(fullDF)
    print("reduction = " + str(reduction))
    # Default pk-score for if a possible characteristics value has no mutants in selection.
    defaultPK = len(sampleDF[sampleDF["killed"] == 1]) / len(sampleDF)

    trainingInsert = pkScoreTransformer(fullDF, sampleDF, defaultPK)

    # Saves the results of the "executed" mutants to overwrite the predictions.
    if useSampled:
        killedIndices = fullDF.index[(fullDF["killed"] == 1) & (fullDF["id"].isin(sampleMutantsIDs))].tolist()
        survivedIndices = fullDF.index[(fullDF["killed"] == 0) & (fullDF["id"].isin(sampleMutantsIDs))].tolist()
    fullDF = fullDF.drop(columns=["id", "killed"])

    predictResults = classifier.predict(fullDF)

    # For the executed mutants, CPMT would not need to predict their result.
    if useSampled:
        predictResults[killedIndices] = 1
        predictResults[survivedIndices] = 0
    print("Prediction for project " + project + " took: " + str(time.time() - start_time) + " seconds.")
    return predictResults, reduction


# Calculates the precision: percentage of correct predictions in decimals.
def precisionCalc(project: str, predictions: np.ndarray) -> float:
    dataframes = getProjectDfs(project)
    if dataframes == -1:
        return -1
    data, results = dataframes
    data["prediction"] = predictions
    newData = data[["id", "prediction"]]
    merged = newData.merge(results, how="inner", on="id")
    merged.to_csv("projects/" + project + "/predictions.csv", sep=",", index=False)
    correctList = [1 if i == j else 0 for i, j in zip(merged["killed"].tolist(), merged["prediction"].tolist())]
    precision = len([1 for val in correctList if val == 1 ])/len(correctList)
    print("Precision = " + str(precision))
    return precision

Parameter cell down below. Note that these are not the only parameters you might want to change.

In [ ]:
seeds = [
    66304, 16389, 14706, 91254, 49890, 86054, 55284, 77324, 36147, 13506, 73920, 80157, 43981, 75358, 33399, 56134,
    13388, 81617, 90957, 52113, 20428, 26482, 56340, 31018, 32067, 13067, 8339, 49008, 125894, 68282, ]
# trainingProjects = ["google-auto-service", "scribejava-core", "commons-cli",
#                     "google-auto-value",] # "gson", "commons-io", "commons-codec"
projects = ["google-auto-common", "scribejava-core", "google-auto-factory", "commons-csv",
                "commons-cli", "google-auto-value", "gson", "commons-io","commons-text", "commons-codec", ]
parameters = [0.05, 5, 3, 3, 3, 3, 2, 2, 2]
# classifier = SGDClassifier()
reduction = [0.1]

In [ ]:
warnings.filterwarnings('ignore')

Below is the cell with CPMT experiments using cluster center selection for the mutant selection.

In [ ]:
# Hyperopt was not used for ClusterSelection's parameters or prepParams.
# We instead used cmtImprovement's values for prepParams and some basic
# values for parameters, these are default arguments of the functions.
def tryOut(args):
    # parameters = [args["localityReduction"], args["n_localVarsClusters"], args["perOperator"], args["perOpcode"], args["perReturn"],
    #               args["perTryCatch"], args["perLocCluster"], args["perVarsCluster"], args["samplingStop"]]
    classAlgo = LinearSVC(dual=False, tol=args["tol"], C=args["c"])
    precisions = []
    reductions = []
    for i in range(len(projects)):
        predictProject = projects.pop(i)
        try:
            classifier = trainAlgorithm(projects, classAlgo)
        except Exception:
            traceback.print_exc()
        results, reduction = own_predict(classifier, predictProject)
        precisions.append(precisionCalc(predictProject, results))
        reductions.append(reduction)
        projects.insert(i, predictProject)
    performance = [np.mean(reductions), np.mean(precisions), np.std(precisions), max(precisions), min(precisions)]
    print(performance)

tryOut({'c': 0.498087050393805, 'localityReduction': 0.01582020546979156, 'n_localVarsClusters': 2, 'perLocCluster': 1, 'perOpcode': 27, 'perOperator': 10, 'perReturn': 17, 'perTryCatch': 4, 'perVarsCluster': 22, 'samplingStop': 0.1537291955327681, 'tol': 0.005117505124731138})